# Feature Engineering for Demographic 'B'

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

### Read in relevant data

In [2]:
Block = 'B'
df = pd.read_csv(r"C:\Users\gebruiker\Desktop\Thesis Scripts\Blocks\B\ACORN-B.csv", sep="\t")
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('Unnamed: 0.1', axis=1, inplace=True)
df.drop('Acorn_grouped', axis=1, inplace=True)

c:\users\gebruiker\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Import weather data and drop Unnamed column
weather = pd.read_csv(r'C:\Users\gebruiker\Desktop\Thesis Scripts\Data\weather_engineered.csv', sep="\t")
weather.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
# Read in holiday data
holiday = pd.read_csv(r'C:\Users\gebruiker\Desktop\Thesis Scripts\Data\holiday.csv', sep='\t', encoding='utf-8')
holiday.drop('Unnamed: 0', axis=1, inplace=True)
# Subset relevant dates for the present dataset
holiday = holiday[((holiday.Month == 12) & (holiday.Year == 2012)) | (holiday.Year == 2013) | ((holiday.Month == 1) & (holiday.Year == 2014))]

### Define Functions

In [5]:
def clean_and_weather(df):
    colnames = list(df.columns.values)
    df = df.values
    new_df = []
    for i in range(len(df)):
        if df[i][9] < 5:
            df[i][9] = 0
        elif df[i][9] >= 5:
            df[i][9] = 1
        if df[i][0] in Useful_IDs:
            new_df.append(df[i])
    df = pd.DataFrame(data=new_df, columns=colnames)
    return df

In [6]:
def Seasonality(df):
    # Initialize list for seasons data
    lst_seasons = []
    
    # Append season corresponding to Month number
    for i in range(len(df)):
        if df['Month'].values[i] in [1,2,12]:
            lst_seasons.append('Winter')
        elif df['Month'].values[i] in [3,4,5]:
            lst_seasons.append('Spring')
        elif df['Month'].values[i] in [6,7,8]:
            lst_seasons.append('Summer')
        elif df['Month'].values[i] in [9,10,11]:
            lst_seasons.append('Autumn')
    # Convert list to Pandas DF
    seasons = pd.DataFrame(lst_seasons)
    
    # Merge seasons to df and rename the seasons column to 'Seasons'
    df = pd.concat([df.reset_index(drop=True), seasons.reset_index(drop=True)], axis = 1)
    df.rename(columns={0: 'Season'}, inplace = True)
    
    return df

In [7]:
def zero_padding(test):
    list_hour = []
    for i in range(len(test)):
        x = str(test.iloc[i]).zfill(2)
        list_hour.append(x)
    return list_hour

In [8]:
def isHoliday(df, holiday):
    # Merge dataset while keeping the shape of the original dataset
    df = pd.merge(df, holiday, how='outer', on=['Year', 'Month', 'Day'])
    # Fill na's with 0's
    df.fillna(0, inplace=True)
    # Replace holidays with 1
    colnames = list(df.columns.values)
    df = df.values
    # Replace holidays with 1
    for i in range(len(df)):
        if len(str(df[i][23])) > 1:
            df[i][23] = 1

    df = pd.DataFrame(data=df, columns=colnames)
    # Rename column Type to isHoliday
    df.rename(columns={'Type': 'isHoliday'}, inplace = True)
    return df

### Provide Summaries of data

#### Tweak the Useful IDS. Now for prediction's sake it is set to very extreme values.

In [9]:
# Save distribution of MACIDs
ID_counts = df['LCLid'].value_counts() 
# Save the useful IDs relevant to out analysis
Useful_IDs = list(ID_counts[ID_counts >= (ID_counts.max()* 1)].index)
# Return the ratio of included to excluded Smart Meters
len(Useful_IDs)/len(ID_counts)

0.2

In [10]:
df_c = clean_and_weather(df)
# Return the ratio of included observations from the original dataset
df_c.shape[0]/df.shape[0]

0.20775366351326285

In [11]:
# Included smare meters
len(set(list(df_c['LCLid'])))

5

In [12]:
# Save distribution of weekdays
weekday_counts = df_c['Weekday#'].value_counts() 

# The meters report data 99.7% of all the collection moments at a half-hourly level
weekdays = weekday_counts[0] / sum(weekday_counts)
print(weekdays / (5/7))

1.0017857142857143


### Combine datasets - weather and holidays

In [13]:
df_c['Hour'][df_c['Minute'] == 30] += .5

c:\users\gebruiker\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
# Merge df and weather data by 'Year', 'Month', 'Day', 'Hour'
new_df = pd.merge(df_c, weather, how='outer', on=['Year', 'Month', 'Day', 'Hour'])
df = new_df.iloc[:,0:23]

# Clear redundant data from memory
del new_df, weather, df_c

In [15]:
# Apply Seasonality function to df
df = Seasonality(df)

In [16]:
df.columns

Index(['LCLid', 'stdorToU', 'KWH/hh (per half hour) ', 'Acorn', 'Year',
       'Month', 'Day', 'Hour', 'Minute', 'Weekday#', 'apparentTemperature',
       'cloudCover', 'dewPoint', 'humidity', 'pressure', 'temperature',
       'visibility', 'windBearing', 'windSpeed', 'Season'],
      dtype='object')

In [17]:
# Create dummies for seasons and binary for Standard or Time-of-Use
df = pd.get_dummies(df, columns=['Season', 'stdorToU'])
df.rename(columns={'stdorToU_Std': 'StdorToU'}, inplace=True)

In [18]:
# Confirm what dates are included
holiday

,Type,Year,Month,Day
0,Boxing Day,2012,12,26
1,Christmas Day,2012,12,25
9,Boxing Day,2013,12,26
10,Christmas Day,2013,12,25
11,Summer bank holiday,2013,8,26
12,Spring bank holiday,2013,5,27
13,Early May bank holiday,2013,6,5
14,Easter Monday,2013,1,4
15,Good Friday,2013,3,29
16,New Year?s Day,2013,1,1


In [19]:
# Merge holiday data into isHoliday column. 1 for is holiday, 0 else.
df = isHoliday(df, holiday)

# Remove wrong data inputs for KWH/halfh

In [20]:
df.rename(columns={'KWH/hh (per half hour) ': 'KWH_hh'}, inplace=True)
df['KWH_hh'] = pd.to_numeric(df['KWH_hh'], errors='coerce').fillna(-100)
len(set(list(df["LCLid"])))

6

In [21]:
df[df['KWH_hh'] == -100]
len(set(list(df['LCLid'][df['KWH_hh'] == -100])))

5

In [22]:
# Remove fucked up observations
df = df[df['KWH_hh'] != -100]
#df = df[df['KWH/halfh'] != "Null"]

### Add Date columns

In [23]:
def datetime_to_string(df):
    vals = df.values
    print(df.columns)
    # Convert column values to integers (rounds numbers down) e.g 0.9 -> 0
    vals[:,2] = [int(vals[i,3]) for i in range(len(vals[:]))]
    vals[:,3] = [int(vals[i,4]) for i in range(len(vals[:]))]
    vals[:,4] = [int(vals[i,5]) for i in range(len(vals[:]))]
    vals[:,5] = [int(vals[i,6]) for i in range(len(vals[:]))]
    vals[:,6] = [int(vals[i,7]) for i in range(len(vals[:]))]
    
    datetime = ['{:%Y-%m-%d %H:%M}'.format(dt.datetime(vals[i,2],vals[i,3],vals[i,4],
                                                       vals[i,5],vals[i,6])) for i in range(len(vals[:]))]
    df['DateTime'] = datetime
    return df

In [24]:
df = datetime_to_string(df)

Index(['LCLid', 'KWH_hh', 'Acorn', 'Year', 'Month', 'Day', 'Hour', 'Minute',
       'Weekday#', 'apparentTemperature', 'cloudCover', 'dewPoint', 'humidity',
       'pressure', 'temperature', 'visibility', 'windBearing', 'windSpeed',
       'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
       'StdorToU', 'isHoliday'],
      dtype='object')


### Create time categories

In [25]:
# Initialize column with value None
df['TimePeriod'] = 'Peak'
# Implement categories for Time period of the day
df['TimePeriod'][(df['Hour'] >= 8) & (df['Hour'] < 16)] = 'Off-peak'
df['TimePeriod'][(df['Hour'] >= 0) & (df['Hour'] < 8)] = 'Night'

c:\users\gebruiker\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
c:\users\gebruiker\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [26]:
len(set(list(df['Hour'][df['TimePeriod']=='Morning'])))

0

In [27]:
len(set(list(df['Hour'][df['TimePeriod']=='Evening'])))

0

In [28]:
len(set(list(df['Hour'][df['TimePeriod']=='Night'])))

16

In [29]:
# Create dummy variable for Morning, Evening, Night
df = pd.get_dummies(df, columns=['TimePeriod'])

In [30]:
df.columns

Index(['LCLid', 'KWH_hh', 'Acorn', 'Year', 'Month', 'Day', 'Hour', 'Minute',
       'Weekday#', 'apparentTemperature', 'cloudCover', 'dewPoint', 'humidity',
       'pressure', 'temperature', 'visibility', 'windBearing', 'windSpeed',
       'Season_Autumn', 'Season_Spring', 'Season_Summer', 'Season_Winter',
       'StdorToU', 'isHoliday', 'DateTime', 'TimePeriod_Night',
       'TimePeriod_Off-peak', 'TimePeriod_Peak'],
      dtype='object')

### Final touch-ups

In [31]:
# Rename columns for nicer names
df.rename(columns={'stdorToU_ToU': 'StdorToU', 'Weekday#': 'isWeekend',
                   'TimePeriod_Peak': 'isPeak','TimePeriod_Off-peak': 'isOff_Peak',
                   'TimePeriod_Night': 'isNight', 'Season_Autumn': 'isAutumn', 'Season_Spring': 'isSpring',
                   'Season_Summer': 'isSummer', 'Season_Winter': 'isWinter'
                  }, inplace=True)

In [32]:
# Change is weekend values from float to integer
df['isWeekend'] = df['isWeekend'].astype(int)

In [33]:
# Change order of columns
cols = ['LCLid', 'Acorn', 'Year', 'Month', 'Day', 'Hour', 'DateTime',
       'apparentTemperature', 'cloudCover', 'dewPoint', 'humidity', 'pressure',
       'temperature', 'visibility', 'windBearing', 'windSpeed', 'StdorToU', 'isAutumn',
       'isSpring', 'isSummer', 'isWinter', 'isHoliday',
       'isPeak', 'isOff_Peak', 'isNight', 'isWeekend', 'KWH_hh']
df = df[cols]

In [34]:
df.head()

,LCLid,Acorn,Year,Month,Day,Hour,DateTime,apparentTemperature,cloudCover,dewPoint,...,isAutumn,isSpring,isSummer,isWinter,isHoliday,isPeak,isOff_Peak,isNight,isWeekend,KWH_hh
0,MAC001966,ACORN-B,2012,11,1,0,2012-11-01 00:00,11.1933,0.602,46.82,...,1,0,0,0,0,0,0,1,0,0.162
1,MAC003355,ACORN-B,2012,11,1,0,2012-11-01 00:00,11.1933,0.602,46.82,...,1,0,0,0,0,0,0,1,0,0.248
2,MAC003445,ACORN-B,2012,11,1,0,2012-11-01 00:00,11.1933,0.602,46.82,...,1,0,0,0,0,0,0,1,0,0.507
3,MAC003876,ACORN-B,2012,11,1,0,2012-11-01 00:00,11.1933,0.602,46.82,...,1,0,0,0,0,0,0,1,0,0.139
4,MAC004567,ACORN-B,2012,11,1,0,2012-11-01 00:00,11.1933,0.602,46.82,...,1,0,0,0,0,0,0,1,0,0.054


In [37]:
# View subset of data
df[df['LCLid'] == 'MAC001966']

,LCLid,Acorn,Year,Month,Day,Hour,DateTime,apparentTemperature,cloudCover,dewPoint,...,isAutumn,isSpring,isSummer,isWinter,isHoliday,isPeak,isOff_Peak,isNight,isWeekend,KWH_hh
0,MAC001966,ACORN-B,2012,11,1,0,2012-11-01 00:00,11.1933,0.602,46.82,...,1,0,0,0,0,0,0,1,0,0.162
5,MAC001966,ACORN-B,2012,11,1,0.5,2012-11-01 00:30,10.1794,0.677,47.043,...,1,0,0,0,0,0,0,1,0,0.111
10,MAC001966,ACORN-B,2012,11,1,1,2012-11-01 01:00,9.16556,0.752,47.266,...,1,0,0,0,0,0,0,1,0,0.115
15,MAC001966,ACORN-B,2012,11,1,1.5,2012-11-01 01:30,7.54278,0.754,46.453,...,1,0,0,0,0,0,0,1,0,0.109
20,MAC001966,ACORN-B,2012,11,1,2,2012-11-01 02:00,5.92,0.756,45.64,...,1,0,0,0,0,0,0,1,0,0.145
25,MAC001966,ACORN-B,2012,11,1,2.5,2012-11-01 02:30,5.84778,0.758,45.402,...,1,0,0,0,0,0,0,1,0,0.166
30,MAC001966,ACORN-B,2012,11,1,3,2012-11-01 03:00,5.77556,0.76,45.164,...,1,0,0,0,0,0,0,1,0,0.151
35,MAC001966,ACORN-B,2012,11,1,3.5,2012-11-01 03:30,5.70167,0.674,44.355,...,1,0,0,0,0,0,0,1,0,0.150
40,MAC001966,ACORN-B,2012,11,1,4,2012-11-01 04:00,5.62778,0.588,43.546,...,1,0,0,0,0,0,0,1,0,0.104
45,MAC001966,ACORN-B,2012,11,1,4.5,2012-11-01 04:30,5.56944,0.552,43.552,...,1,0,0,0,0,0,0,1,0,0.113


### Save data to csv for modelling

In [36]:
# Save df to working directory
df.to_csv('Feat_Eng_'+Block+'.csv', sep='\t', encoding='utf-8')